In [26]:
import os 
import glob 

from shutil import copyfile, rmtree
import json

def acopyjson(a, c):
    with open(a, "r") as f:
        src = json.load(f)
    
    dst = {}
    if os.path.isfile(c):
        with open(c, "r") as f:
            dst = json.load(f)
        
    ctr = len(dst.keys())
    # print(f'before {len(dst.keys()), len(src.keys())}')
    for k, v in src.items():
        dst[ctr] = v
        ctr += 1
    
    # print(f'after {len(dst.keys())}')
    with open(c, "w") as fp:
        json.dump(dst, fp)

def acopydir(a, c, ext):
    if not os.path.isdir(c):
        os.makedirs(c)
    
    fsa = [x for x in glob.glob(os.path.join(a, f'*{ext}'))]
    # print(f'copying {len(fsa)} files from {a} into {c}')
    
    ctr = len(glob.glob(os.path.join(c, f'*{ext}')))
    # print(f'{ctr} files already exist')
    
    for f in fsa:
        fname = "{:05d}{}".format(ctr, ext)
        # print(f'copyfile {fname, os.path.join(c, fname)}')
        copyfile(f, os.path.join(c, fname))
        ctr += 1
    
#     print(f'acopydir {len(fsa), len(fsb)}')
    
    
def combine(ex_list, traj_ab):
    if not os.path.isdir(traj_ab):
        os.makedirs(traj_ab)
    
    # copy over rgb, depth, seg from traj a
    # copy over rgb, d, seg from traj b but with new names 
    for x in ex_list:
        acopydir(os.path.join(x, 'rgb'), os.path.join(traj_ab, 'rgb'), '.jpg')  
        acopydir(os.path.join(x, 'rgb_dbg'), os.path.join(traj_ab, 'rgb_dbg'), '.jpg')  
        acopydir(os.path.join(x, 'depth'), os.path.join(traj_ab, 'depth'), '.npy')
        acopydir(os.path.join(x, 'seg'), os.path.join(traj_ab, 'seg'), '.npy')
        acopyjson(os.path.join(x, 'data.json'), os.path.join(traj_ab, 'data.json'))
    

def combine_all_examines_for_traj(src, dst):
    examines = [os.path.join(src, x) for x in os.listdir(src) if x.isdigit()]
    
    if len(examines) > 1:
        print(f'combining {len(examines)} examines for {src} into {dst}')
        combine(examines, dst)
        
        # do basic sanity check
        basic_sanity(dst) 


def basic_sanity(traj_path):
    def ll(x, y, ext):
        l = len(glob.glob(os.path.join(x, y, f'*{ext}')))
        # print(f'len({x}/{y}) {l}')
        return l
    # load json
    with open(os.path.join(traj_path, 'data.json'), 'r') as f:
        data = json.load(f)
        # print(f'data.json {len(data.keys())}')
    assert ll(traj_path, 'rgb', '.jpg') == ll(traj_path, 'seg', '.npy') == ll(traj_path, 'depth', '.npy') == len(data.keys()), "basic sanity check failed!!"        

def combine_examines(root_dir, out_dir):
    if not os.path.isdir(out_dir):
        os.makedirs(out_dir)
    
    for x in os.listdir(root_dir):
        if x.isdigit():
            # this is a trajectory that needs all examines combined 
            traj_path = os.path.join(root_dir, x)
            combine_all_examines_for_traj(
                traj_path, 
                os.path.join(out_dir, x)
            )
            
rd = '/checkpoint/apratik/data_dec/'

og_list = [
    'circle', 
    'circle_noisy2',
    'baseline', 
    'baseline_noisy2',
    'straightline', 
    'st_noisy2',
]

for x in og_list:
    cur_dir = os.path.join(rd, x)
    out_dir = os.path.join(rd, 'combined_examines', x)
    print(cur_dir, out_dir)
    print(f'recreating {out_dir}')
    rmtree(out_dir)
    combine_examines(cur_dir, out_dir)

/checkpoint/apratik/data_dec/circle /checkpoint/apratik/data_dec/combined_examines/circle
recreating /checkpoint/apratik/data_dec/combined_examines/circle
combining 19 examines for /checkpoint/apratik/data_dec/circle/0 into /checkpoint/apratik/data_dec/combined_examines/circle/0
combining 21 examines for /checkpoint/apratik/data_dec/circle/2 into /checkpoint/apratik/data_dec/combined_examines/circle/2
combining 31 examines for /checkpoint/apratik/data_dec/circle/5 into /checkpoint/apratik/data_dec/combined_examines/circle/5
combining 25 examines for /checkpoint/apratik/data_dec/circle/6 into /checkpoint/apratik/data_dec/combined_examines/circle/6
combining 42 examines for /checkpoint/apratik/data_dec/circle/8 into /checkpoint/apratik/data_dec/combined_examines/circle/8
combining 24 examines for /checkpoint/apratik/data_dec/circle/11 into /checkpoint/apratik/data_dec/combined_examines/circle/11
combining 22 examines for /checkpoint/apratik/data_dec/circle/15 into /checkpoint/apratik/dat

In [ ]:
a# root_dir = '/checkpoint/apratik/data/data/apartment_0/default/no_noise/mul_traj_200'
root_dir = '/checkpoint/apratik/data/data/apartment_0/default/noise/mul_traj_200'
combine = 2
new_dir = f'/checkpoint/apratik/data/data/apartment_0/default/noise/mul_traj_200_combined_{combine}'

if not os.path.isdir(new_dir):
    os.makedirs(new_dir)
    
# check valid trajectories
def basic_sanity(traj_path):
    def ll(x, y, ext):
        return len(glob.glob(os.path.join(x, y, f'*{ext}')))
    # TODO: add base_pose sanity check here
    return ll(traj_path, 'rgb', '.jpg') == ll(traj_path, 'seg', '.npy') == ll(traj_path, 'depth', '.npy')

valid_traj = []
for x in range(201):
    traj_path = os.path.join(root_dir, str(x))
    if os.path.isdir(traj_path) and basic_sanity(traj_path):
        valid_traj.append(traj_path)

In [2]:
from shutil import copyfile
import json

def acopyjson(a, b, c):
    with open(a, "r") as f:
        dat = json.load(f)
        
    with open(b, "r") as f:
        dat2 = json.load(f)
        
    ctr = len(dat.keys())
#     print(f'before {len(dat.keys()), len(dat2.keys())}')
    for k, v in dat2.items():
        dat[ctr] = v
        ctr += 1
    
#     print(f'after {len(dat.keys())}')
    with open(c, "w") as fp:
        json.dump(dat, fp)

def acopydir(a, b, c, ext):
    if not os.path.isdir(c):
        os.makedirs(c)
    
    fsa = [x for x in glob.glob(os.path.join(a, f'*{ext}'))]
    
    ctr = 0
    for f in fsa:
        fname = f.split('/')[-1]
        copyfile(f, os.path.join(c, fname))
        ctr += 1

    fsb = [x for x in glob.glob(os.path.join(b, f'*{ext}'))] 
    for f in fsb:
        fname = "{:05d}{}".format(ctr, ext)
        ctr += 1
        copyfile(f, os.path.join(c, fname))
    
#     print(f'acopydir {len(fsa), len(fsb)}')
    
    
def combine(traj_a, traj_b, traj_ab):
    if not os.path.isdir(traj_ab):
        os.makedirs(traj_ab)
    
    # copy over rgb, depth, seg from traj a
    # copy over rgb, d, seg from traj b but with new names 
    acopydir(os.path.join(traj_a, 'rgb'), os.path.join(traj_b, 'rgb'), os.path.join(traj_ab, 'rgb'), '.jpg')   
    acopydir(os.path.join(traj_a, 'depth'), os.path.join(traj_b, 'depth'), os.path.join(traj_ab, 'depth'), '.npy')
    acopydir(os.path.join(traj_a, 'seg'), os.path.join(traj_b, 'seg'), os.path.join(traj_ab, 'seg'), '.npy')
    acopyjson(os.path.join(traj_a, 'data.json'), os.path.join(traj_b, 'data.json'), os.path.join(traj_ab, 'data.json'))
    
ctr = 1
for x in range(0, len(valid_traj), 2):
    a = valid_traj[x]
    b = valid_traj[x+1]
    if os.path.isdir(a) and os.path.isdir(b):
        print(f'combining {a} and {b}')
        combine(a, b, os.path.join(new_dir, str(ctr)))
        ctr += 1
    

combining /checkpoint/apratik/data/data/apartment_0/default/noise/mul_traj_200/1 and /checkpoint/apratik/data/data/apartment_0/default/noise/mul_traj_200/2
combining /checkpoint/apratik/data/data/apartment_0/default/noise/mul_traj_200/3 and /checkpoint/apratik/data/data/apartment_0/default/noise/mul_traj_200/4
combining /checkpoint/apratik/data/data/apartment_0/default/noise/mul_traj_200/5 and /checkpoint/apratik/data/data/apartment_0/default/noise/mul_traj_200/6
combining /checkpoint/apratik/data/data/apartment_0/default/noise/mul_traj_200/7 and /checkpoint/apratik/data/data/apartment_0/default/noise/mul_traj_200/8
combining /checkpoint/apratik/data/data/apartment_0/default/noise/mul_traj_200/9 and /checkpoint/apratik/data/data/apartment_0/default/noise/mul_traj_200/10
combining /checkpoint/apratik/data/data/apartment_0/default/noise/mul_traj_200/11 and /checkpoint/apratik/data/data/apartment_0/default/noise/mul_traj_200/12
combining /checkpoint/apratik/data/data/apartment_0/default/n